In [9]:
import os,socket
import cv2
import datetime
import numpy as np

from matplotlib.ticker import EngFormatter
efmt = EngFormatter('B')

fext = ".png"
fpath = "model.q.3.pinn"
froot = os.path.join(os.getcwd(),fpath)
fnames = [os.path.join(froot,fname) for fname in os.listdir(froot) if fname.endswith(fext)]

print(f"author  {socket.gethostname():>26}")
print(f"version {str(datetime.datetime.now()):>26}")

print(f"model   {fpath:>26}")
print(f"files   {str(len(fnames)):>26}")

author                         NOB
version 2023-06-22 03:51:59.186153
model               model.q.3.pinn
files                           93


### prepare a video mp4 file by monitored images

base generator is tclab.model.animate.3.py   

In [13]:
shape = (3200,1108)
duration = 1 # duration of one slide in s
fps = 24

item_count = len(fnames)

target_time = str(datetime.timedelta(seconds = duration * item_count))
target_size = int(os.path.getsize(fnames[0]) * item_count * fps)

print(f"video time estimated: {target_time}, size (w/o zip) estimated: {efmt(target_size)}")
print(f"{item_count} items, {duration}s duration, {fps} fps")


video time estimated: 0:01:33, size (w/o zip) estimated: 192.865 MB
93 items, 1s duration, 24 fps


In [6]:
%%time 

fformat = 'mp4'
ftarget = os.path.join(froot,f"{fpath}.{fformat}")

if fformat == 'mp4':    
    fourcc = cv2.VideoWriter.fourcc(*'MP4V')
else:
    fourcc = cv2.VideoWriter.fourcc(*'XVID')

writer = cv2.VideoWriter(ftarget, fourcc, fps, shape)

print(f"video target {ftarget}")

for fname in fnames[:item_count]:

    src = cv2.imread(fname)

    height, width, _ = src.shape
    scale = min(shape[0]/width, shape[1]/height)
    size = (int(width*scale), int(height*scale))
    src = cv2.resize(src, size)

    dst = np.zeros((shape[1], shape[0], 3), dtype='uint8') 
    
    xoff = (shape[0] - size[0]) // 2
    yoff = (shape[1] - size[1]) // 2
    
    dst[yoff:yoff+size[1], xoff:xoff+size[0]] = src
        
    for _ in range(duration * fps):
        writer.write(dst)

writer.release()
cv2.destroyAllWindows()

file_size = os.path.getsize(ftarget)
file_name = os.path.basename(ftarget)

print(f"{file_name}: {efmt(file_size)}")

video target c:\Develop\vsc\tclab\media\model.q.3.pinn\model.q.3.pinn.mp4
model.q.3.pinn.mp4: 29.1485 MB
CPU times: total: 38.4 s
Wall time: 51.9 s
